In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Q"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명 !!!! 반드시 호수 체크!!!!
model_name = "5호모델_{}_k{}_{}".format(freq, num_k, sc)
model_name

'5호모델_Q_k3_new_rank'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트_전체시장.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트_전체시장.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트_전체시장.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액_전체시장"
                                 ,plus_two="CB금액_전체시장", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '유상증자금액_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장', '특수관계인에대한자금대여금액_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액_전체시장+CB금액_전체시장"
                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장', '특수관계인에대한자금대여금액_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장', '특수관계인에대한자금대여금액_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액_전체시장"
                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트_전체시장"
                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송_전체시장"
                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kospi6 = tools.dict_data_plus(concat_kospi5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kosdaq6 = tools.dict_data_plus(concat_kosdaq5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장', '상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '불성실공시법인지정카운트_전체시장',
       '조회공시요구카운트_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
                                 ,plus_two="불성실공시법인지정카운트_전체시장", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '조회공시요구카운트_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
concat_together8 = tools.dict_data_plus(concat_together7,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장"
                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together8["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
concat_together9 = tools.dict_data_plus(concat_together8,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장"
                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together9["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [18]:
concat_together10 = tools.dict_data_plus(concat_together9,
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장"
                                 ,plus_two="타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together10["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [19]:
concat_together11 = tools.dict_data_plus(concat_together10,
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
                                 ,plus_two="BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장", return_origianl=False)
print(concat_together11["2012-01"].columns.shape)
concat_together11["2012-01"].columns

(14,)


Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [20]:
drop_togehter = tools.dict_data_drop(concat_together8.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

scaled_together["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [21]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=50, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 50/50 [00:03<00:00, 14.17it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 48,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=48)},
 '2012-04': {'num_of_cluseer': 3,
  'random_state': 1,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=1)},
 '2012-07': {'num_of_cluseer': 3,
  'random_state': 15,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=15)},
 '2012-10': {'num_of_cluseer': 3,
  'random_state': 36,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=36)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 22,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=22)},
 '2013-04': {'num_of_cluseer': 3,
  'random_state': 11,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=11)},
 '2013-07': {'num_of_cluseer': 3,
  'random_state': 49,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=49)},
 '2013-10': {'num_of_cluseer': 3,
  'random_state': 28,
  'model': KMeans(max_iter=50

## 모델기록


In [22]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

796.0
455.0


,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2020-04,2020-07,2020-10,2021-01,2021-04,2021-07,2021-10,2022-01,2022-04,2022-07
0,614.0,504.0,552.0,575.0,511.0,593.0,517.0,506.0,533.0,535.0,...,696.0,551.0,558.0,707.0,700.0,657.0,727.0,574.0,578.0,641.0
1,525.0,594.0,472.0,545.0,489.0,517.0,495.0,600.0,590.0,500.0,...,550.0,740.0,767.0,739.0,634.0,669.0,766.0,740.0,724.0,796.0
2,455.0,506.0,583.0,488.0,605.0,495.0,588.0,510.0,495.0,583.0,...,730.0,708.0,689.0,579.0,709.0,730.0,562.0,742.0,745.0,602.0
기업수,1594.0,1604.0,1607.0,1608.0,1605.0,1605.0,1600.0,1616.0,1618.0,1618.0,...,1976.0,1999.0,2014.0,2025.0,2043.0,2056.0,2055.0,2056.0,2047.0,2039.0


In [23]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

eval_one_original

,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2018-10,2019-01,2019-04,2019-07,2019-10,2020-01,2020-04,2020-07,2020-10,2021-01
0,0.016092,0.105747,0.004598,0.004598,0.006897,0.006897,0.002299,0.004598,0.080460,0.094253,...,0.002299,0.000000,0.022989,0.004598,0.002299,0.020690,0.181609,0.004598,0.006897,0.016092
1,0.098851,0.011494,0.078161,0.006897,0.073563,0.064368,0.075862,0.009195,0.013793,0.002299,...,0.197701,0.193103,0.006897,0.025287,0.022989,0.177011,0.002299,0.167816,0.149425,0.128736
2,0.006897,0.011494,0.009195,0.071264,0.000000,0.004598,0.009195,0.082759,0.004598,0.011494,...,0.011494,0.011494,0.197701,0.197701,0.170115,0.002299,0.011494,0.006897,0.006897,0.004598
포착군집,1.000000,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,...,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000


In [24]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))
eval_two_original

,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2018-10,2019-01,2019-04,2019-07,2019-10,2020-01,2020-04,2020-07,2020-10,2021-01
0,0.000000,0.063291,0.000000,0.003165,0.003165,0.006329,0.000000,0.000000,0.060127,0.082278,...,0.003165,0.003165,0.025316,0.000000,0.000000,0.012658,0.104430,0.003165,0.003165,0.022152
1,0.056962,0.000000,0.044304,0.003165,0.041139,0.050633,0.063291,0.009494,0.009494,0.000000,...,0.161392,0.164557,0.000000,0.015823,0.012658,0.120253,0.003165,0.094937,0.098101,0.120253
2,0.000000,0.003165,0.003165,0.041139,0.003165,0.000000,0.006329,0.056962,0.000000,0.003165,...,0.012658,0.012658,0.196203,0.126582,0.123418,0.000000,0.018987,0.015823,0.018987,0.006329
포착군집,1.000000,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,...,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000


In [25]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))
eval_three_original

,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2018-10,2019-01,2019-04,2019-07,2019-10,2020-01,2020-04,2020-07,2020-10,2021-01
0,0.019139,0.071770,0.000000,0.014354,0.014354,0.014354,0.004785,0.014354,0.038278,0.038278,...,0.019139,0.014354,0.004785,0.023923,0.019139,0.019139,0.095694,0.023923,0.019139,0.004785
1,0.043062,0.009569,0.071770,0.014354,0.028708,0.028708,0.014354,0.004785,0.014354,0.014354,...,0.129187,0.105263,0.023923,0.009569,0.019139,0.100478,0.019139,0.076555,0.062201,0.062201
2,0.009569,0.009569,0.019139,0.043062,0.019139,0.004785,0.009569,0.009569,0.004785,0.019139,...,0.004785,0.004785,0.095694,0.105263,0.105263,0.023923,0.028708,0.009569,0.004785,0.009569
포착군집,1.000000,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000


In [26]:
evaluate.calculate_catch_probability(eval_one, eval_two, eval_three)

0.8001722339634556

In [27]:
evaluate.calculate_match_probability(eval_one, eval_two, eval_three)

0.8378378378378378

In [28]:
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, eval_three_original)

0.09589929462772855

In [29]:
dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)#.sort_values("2013-01", axis=1, ascending=False)#.to_csv("2호_4_Q_거리.csv")
dist2 = dist.mean().sort_values(ascending=True)
dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))
dist2.sort_values(ascending=False)

매출총이익                                                           0.318104
총자본                                                             0.274874
영업이익                                                            0.255696
당기순이익                                                           0.208072
매출채권                                                            0.204319
판매 및 관리비                                                        0.199550
영업현금흐름                                                          0.166073
최대주주비율                                                          0.145415
BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장                                 0.051138
경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장    0.049702
차입금의존도                                                          0.044449
결산기간_주가변동률                                                      0.039951
최대주주비율_변동성                                                      0.019265
결산기간_거래량증가                                         

In [30]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.142857,0.85,0.055556,0.000000,0.060606,0.105263,0.047619,0.069767,0.872340,0.837209,...,0.131313,0.011765,0.011494,0.058824,0.910256,0.042254,0.030769,0.102564,0.133333,0.0
1,0.767857,0.05,0.861111,0.085714,0.848485,0.868421,0.880952,0.069767,0.106383,0.023256,...,0.010101,0.117647,0.126437,0.929412,0.025641,0.915493,0.892308,0.846154,0.033333,0.0
2,0.089286,0.10,0.083333,0.914286,0.090909,0.026316,0.071429,0.860465,0.021277,0.139535,...,0.858586,0.870588,0.862069,0.011765,0.064103,0.042254,0.076923,0.051282,0.833333,1.0


In [31]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.142857,0.85,0.055556,0.000000,0.060606,0.105263,0.047619,0.069767,0.872340,0.837209,...,0.131313,0.011765,0.011494,0.058824,0.910256,0.042254,0.030769,0.102564,0.133333,0.0
1,0.767857,0.05,0.861111,0.085714,0.848485,0.868421,0.880952,0.069767,0.106383,0.023256,...,0.010101,0.117647,0.126437,0.929412,0.025641,0.915493,0.892308,0.846154,0.033333,0.0
2,0.089286,0.10,0.083333,0.914286,0.090909,0.026316,0.071429,0.860465,0.021277,0.139535,...,0.858586,0.870588,0.862069,0.011765,0.064103,0.042254,0.076923,0.051282,0.833333,1.0


In [32]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [33]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,0.788931,-0.053110,0.263749,2,2.5
A000040,-0.274718,0.310134,-0.404115,1,2.5
A000050,0.773986,-0.039304,0.107833,2,2.5
A000070,1.223623,0.280989,0.062595,2,2.5
A000080,1.156330,0.194850,0.157220,2,2.5
...,...,...,...,...,...
A353810,0.043755,0.023292,0.031959,0,2.5
A354200,-0.949112,-0.142737,0.228610,1,2.5
A357780,0.925722,-0.350717,-0.436046,2,2.5
A363280,0.500367,-0.032427,-0.316187,2,2.5
